In [45]:
import numpy as np
import pandas as pd
import os

In [52]:
data_directory = "../Data/Study/02/"
subdirectory = "recalibrated_calibration/"
fileList = os.listdir(data_directory + subdirectory)

In [53]:
def clean_df(df):
    result = df[df['Movement'] != "start"]
    result = result[result['Movement'] != "transition"]
    result = result[result['EyeTrackingDataValid'] != 'TRUE']
    return result

In [66]:
def calculate_cosine_error(df):
    df['cosineError'] = df.apply(lambda row: np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z']))))), axis=1)
    return df

In [61]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [67]:
def calculate_errors(output_subdirectory):
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            error_df = calculate_euclidean_error(df)
            error_df = calculate_cosine_error(error_df)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.mkdir(os.path.join(data_directory, output_subdirectory))
            error_df_output_path = data_directory + output_subdirectory + '/' + file
            error_df.to_csv(error_df_output_path)
            print(file)
            print("uncalibrated: " + str(error_df['euclideanError'].mean()))
            print("calibrated: " + str(error_df['recalibratedEuclideanError'].mean()))
            print("cosine: " + str(error_df['cosineError'].mean()))
            print("")

In [68]:
calculate_errors("error")

hallway_20230505091641_shifted0_recalibrated_calibration.csv
uncalibrated: 29.009994260278063
calibrated: 26.589934100474718
cosine: 0.48268542685119076

ssWalking_20230505091229_shifted47_recalibrated_calibration.csv
uncalibrated: 59.09522005702312
calibrated: 56.615837715171786
cosine: 1.4778271293608747

wsWalking_20230505094522_shifted16_recalibrated_calibration.csv
uncalibrated: 52.49407321873202
calibrated: 49.98730590293753
cosine: 1.6152727895473873

ssWalking_20230505093314_shifted48_recalibrated_calibration.csv
uncalibrated: 58.76758849409513
calibrated: 55.56258672838841
cosine: 1.516071540172629

ssHeadConstrained_20230505090941_shifted29_recalibrated_calibration.csv
uncalibrated: 4.163990387318883
calibrated: 4.838643080929303
cosine: 0.07200401325505797

ssHeadConstrained_20230505092949_shifted25_recalibrated_calibration.csv
uncalibrated: 5.169504107696743
calibrated: 3.138985807492666
cosine: 0.08822757380330615

calibration_20230505092722_shifted0_recalibrated_calibrati